# Классификация

In [25]:
# импортируем нужные библиотеки и структуры данных
import pandas as pd
from sklearn.model_selection import train_test_split #разделение на выборки
from sklearn.tree import DecisionTreeClassifier #дерево решений
from sklearn.ensemble import RandomForestClassifier #случайный лес
from sklearn.linear_model import LogisticRegression #логистическая регрессия

df = pd.read_csv('users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В данном датасете содержатся данные о клиентах "Мегалайн", которые уже перешли на какой-то из тарифов:

1. Количество звонков за месяц
2. Количество минут за месяц
3. Количество сообщений за месяц
4. Количество потраченных мегабайт за месяц
5. Факт подключения тарифа ultra (1 - подключён ultra, 0 - smart)

Датасет был предоставлен уже предобработанным, поэтому можно приступать к разбиению данных на выборки и работой с моделями.

Перед созданием и работой с моделями, разобьём имеющиеся данные на три выборки: обучающую, валидационную и тестовую. Выбираться они будут в соотношении 60:20:20.

Для начала выделим в датасете признаки и целевой признак, которые будут записаны в переменных features и target соответственно

In [4]:
#выделим обычные признаки и целевые
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

После дважды применим функцию train_test_split библиотеки sklearn. В первый раз в соотношении 80 на 20, где 80 процентов занимают обучающая и валидационная выборки и 20 - тестовая. Во второй раз в соотношении 75 к 25, где 75 процентов придётся на обучающую выборку и 25 на валидационную.

In [5]:
# делим выборки на обучающую+валидационную и тестовую
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.2, 
                                                                            random_state=12189)

# обучаюшую+валидационную делим между собой
features_train, features_valid, target_train, target_valid = train_test_split(features_train, 
                                                                              target_train, 
                                                                              test_size=0.25, 
                                                                              random_state=12189)

# проверяем размеры выборок
print('Размер обучающей выборки:', len(target_train))
print('Размер валидационной выборки:', len(target_valid))
print('Размер тестовой выборки:', len(target_test))

Размер обучающей выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


### Дерево решений

Самая простая модель - это дерево решений. Она обучается быстрее всех вышеперечисленных, но имеет наименьшую точность.

Для того, чтобы создать данную модель, импортируем структуру данных 'DecisionTreeClassifier' из библиотеки sklearn и создадим объект этой структуры.

Точность модели будет проверена при различных значениях (от 1 до 10). Среди результатов будет выбрана наибольшая.

In [26]:
# задаём переменные для определения лучшей модели и лучшего результата точности на валидационной выборке
best_model_tree = None #лучшая модель
best_result_tree = 0 #лучшая точность

# обучаем модель на обучающей выборке и выбираем оптимальную глубину (при которой будет наибольшая точность)
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12189, max_depth=depth) # создаём объект структуры дерева решений
    model_tree.fit(features_train, target_train)
    result = model_tree.score(features_valid, target_valid)
    if result > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result
        
print('Наибольшая точность модели дерева решений на валидационной выборке:', best_result_tree)

Наибольшая точность модели дерева решений на валидационной выборке: 0.8087091757387247


### Случайный лес

Случайный лес по своей сути является объединением нескольких деревьев решений. Обучается намного дольше, потому что работа идёт с несколькими деревьями, но точность будет выше, чем у просто дерева.

Для того, чтобы создать модель случайного леса, импортируем структуру 'RandomForestClassifier' из библиотеки sklearn и создадим объект этой структуры.

Точность модели будет проверена при разном количестве деревьев (10, 20, 30, 40 и 50) с разной глубиной (от 1 до 10). Среди них будет выбрана модель с наибольшим результатом правильных предсказаний на валидационной выборке.



In [29]:
# задаём переменные для определения лучшей модели и лучшего результата точности на валидационной выборке
best_model_forest = None
best_result_forest = 0

# обучаем модель на обучающей выборке и выбираем опттимальную глубину и количество деревьев, которые покажут лучший результат
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model_forest = RandomForestClassifier(random_state=12189, max_depth=depth, n_estimators=est)
        model_forest.fit(features_train, target_train)
        result = model_forest.score(features_valid, target_valid)
        if result > best_result_forest:
            best_model_forest = model_forest
            best_result_forest = result
            
print('Наибольшая точность модели случайного леса на валидационной выборке:', best_result_forest)

Наибольшая точность модели случайного леса на валидационной выборке: 0.8211508553654744


### Логистическая регрессия

Логистическая регрессия по своей скорости обучения и точности будет чем-то средним между деревом решений и случайным лесом.

Для того, чтобы создать модель логистической регрессии, импортируем структуру 'LogisticRegression' из библиотеки sklearn и создадим обхект этой структуры.

У данной модели мало параметров, которые можно менять, поэтому мы проверим её точность только при параметрах по умолчанию.



In [30]:
# обучаем модель на обучающей выборке
best_model_logistic = LogisticRegression(random_state=12189)
best_model_logistic.fit(features_train, target_train)
result = best_model_logistic.score(features_valid, target_valid)

# для удобства сохраняем результаты в других переменных
best_result_logistic = result

print('Точность модели логистической регресси на валидационной выборке:', best_result_logistic)

Точность модели логистической регресси на валидационной выборке: 0.7744945567651633


### Итог

На валидационной выборке лучшую точность показала модель случайного леса. Поэтому проверим её на тестовой выборке.



In [31]:
best_model_forest.score(features_test, target_test)

0.7916018662519441

Точность предсказаний немного упала, но всё равно держится на достаточно высоком уровне.

Проверив работу модели на тестовой выборке, приходим к тому, что лучше всего себя в данной задаче показывает модель случайного леса.